### Alter and update EMPIRE Input - Sets.xlsx

Tab: OffshoreNodes missing in Public-EMPIRE, incl. in OpenEMPIRE

In [36]:
import pandas as pd
import numpy as np

In [37]:
NODES_TO_BE_REMOVED = ["Utsira Nord", "Sørlige Nordsjø I", "Sørlige Nordsjø II"]

#### Tab: Nodes

In [38]:
nodes_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="Nodes")
nodes_tab

,Node
0,Austria
1,Bosnia H
2,Belgium
3,Bulgaria
4,Switzerland
5,Czech R
6,Germany
7,Denmark
8,Estonia
9,Spain


Update with new OW nodes

In [39]:
OW_nodes = pd.read_csv("Output/OW_nodes.csv")
OW_nodes = OW_nodes[["Node"]]
nodes_tab = pd.concat([nodes_tab[~nodes_tab["Node"].isin(NODES_TO_BE_REMOVED)], OW_nodes]).drop_duplicates().reset_index(drop=True)
nodes_tab

,Node
0,Austria
1,Bosnia H
2,Belgium
3,Bulgaria
4,Switzerland
...,...
64,Nordavind C
65,Vestavind E
66,Sørvest D
67,Energyhub North


#### Tab: OffshoreNodes

Need to exclude Energyhubs

In [65]:
ENERGY_HUBS = ["Energyhub EU", "Energyhub North", "Energyhub Central"]

offshore_nodes_tab = OW_nodes[~OW_nodes["Node"].isin(ENERGY_HUBS)].rename(columns={"Node": "OffshoreNode"}) 
offshore_nodes_tab

,OffshoreNode
0,Moray Firth
1,Firth of Forth
2,Dogger Bank
3,Hornsea
4,Outer Dowsing
5,Norfolk
6,East Anglia
7,Borssele
8,Hollandsee Kust
9,Helgoländer Bucht


#### Tab: Storage

Keep as is

#### Tab: Technology 

Keep as is

#### Tab: Generators

Keep as is

#### Tab: LineType

Keep as is

#### Tab: StorageOfNodes

Keep as is

#### Tab: DirectionalLines

In [41]:
directional_lines_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="DirectionalLines", header=2)[["NodeFrom", "NodeTo"]]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeFrom"].isin(NODES_TO_BE_REMOVED)]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeTo"].isin(NODES_TO_BE_REMOVED)].reset_index(drop=True)
directional_lines_tab

,NodeFrom,NodeTo
0,Switzerland,Austria
1,Czech R,Austria
2,Germany,Austria
3,Hungary,Austria
4,Italy,Austria
...,...,...
339,Great Brit.,Energyhub EU
340,Netherlands,Energyhub EU
341,Germany,Energyhub EU
342,Denmark,Energyhub EU


Grab newly generated directional lines

In [42]:
df_dl = pd.read_csv("Output/directional_lines.csv")
df_dl

,FromNode,ToNode,lineLength in km,LineType
0,Moray Firth,Firth of Forth,215.667240,HVDC_Cable
1,Moray Firth,Dogger Bank,509.370695,HVDC_Cable
2,Firth of Forth,Dogger Bank,333.632410,HVDC_Cable
3,Moray Firth,Hornsea,586.957998,HVDC_Cable
4,Firth of Forth,Hornsea,387.595780,HVDC_Cable
...,...,...,...,...
385,Energyhub Central,NO2,309.278517,HVDC_OffshoreToOnshore
386,Energyhub Central,Great Brit.,308.992822,HVDC_OffshoreToOnshore
387,Energyhub Central,Denmark,305.970500,HVDC_OffshoreToOnshore
388,Energyhub Central,Netherlands,327.564532,HVDC_OffshoreToOnshore


In [43]:
offshr_lines = df_dl[(df_dl["LineType"] == "HVDC_Cable") | (df_dl["LineType"] == "HVDC_OffshoreToOnshore")]
offshr_areas = set(offshr_lines["FromNode"].values).union(offshr_lines["ToNode"].values)
offshr_areas

{'Belgium',
 'Borssele',
 'Denmark',
 'Dogger Bank',
 'East Anglia',
 'Energyhub Central',
 'Energyhub EU',
 'Energyhub North',
 'Firth of Forth',
 'Germany',
 'Great Brit.',
 'Helgoländer Bucht',
 'Hollandsee Kust',
 'Hornsea',
 'Moray Firth',
 'NO2',
 'NO3',
 'NO4',
 'NO5',
 'Netherlands',
 'Nordavind A',
 'Nordavind B',
 'Nordavind C',
 'Nordavind D',
 'Nordsøen',
 'Nordvest A',
 'Nordvest B',
 'Nordvest C',
 'Norfolk',
 'Outer Dowsing',
 'Sønnavind A',
 'Sørvest A',
 'Sørvest B',
 'Sørvest C',
 'Sørvest D',
 'Sørvest E',
 'Sørvest F',
 'Vestavind A',
 'Vestavind B',
 'Vestavind C',
 'Vestavind D',
 'Vestavind E',
 'Vestavind F'}

Remove cables in original set that are to be replaced 

In [44]:
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeFrom"].isin(offshr_areas)]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeTo"].isin(offshr_areas)].reset_index(drop=True)
directional_lines_tab

,NodeFrom,NodeTo
0,Switzerland,Austria
1,Czech R,Austria
2,Hungary,Austria
3,Italy,Austria
4,Slovenia,Austria
...,...,...
91,Italy,Slovenia
92,Austria,Switzerland
93,Czech R,Slovakia
94,Hungary,Slovakia


Add newly generated lines (two-way conns)

In [45]:
first_way = df_dl[["FromNode", "ToNode"]].rename(columns={"FromNode": "NodeFrom", "ToNode": "NodeTo"})
second_way = first_way.copy().rename(columns={"NodeFrom": "NodeTo", "NodeTo": "NodeFrom"})
second_way

,NodeTo,NodeFrom
0,Moray Firth,Firth of Forth
1,Moray Firth,Dogger Bank
2,Firth of Forth,Dogger Bank
3,Moray Firth,Hornsea
4,Firth of Forth,Hornsea
...,...,...
385,Energyhub Central,NO2
386,Energyhub Central,Great Brit.
387,Energyhub Central,Denmark
388,Energyhub Central,Netherlands


In [46]:
directional_lines_tab = pd.concat([directional_lines_tab, first_way, second_way], ignore_index=True).reset_index(drop=True)
directional_lines_tab

,NodeFrom,NodeTo
0,Switzerland,Austria
1,Czech R,Austria
2,Hungary,Austria
3,Italy,Austria
4,Slovenia,Austria
...,...,...
871,NO2,Energyhub Central
872,Great Brit.,Energyhub Central
873,Denmark,Energyhub Central
874,Netherlands,Energyhub Central


#### Tab: LineTypeOfDirectionalLines

Same structure as Tab: DirectionalLines but with LineType incl. 

In [47]:
line_type_of_directional_lines_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="LineTypeOfDirectionalLines", header=2)
line_type_of_directional_lines_tab

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Germany,Austria,HVAC_OverheadLine
3,Hungary,Austria,HVAC_OverheadLine
4,Italy,Austria,HVAC_OverheadLine
...,...,...,...
419,NO2,Energyhub EU,HVDC_OffshoreToOnshore
420,Great Brit.,Energyhub EU,HVDC_OffshoreToOnshore
421,Netherlands,Energyhub EU,HVDC_OffshoreToOnshore
422,Germany,Energyhub EU,HVDC_OffshoreToOnshore


In [48]:
# Output format with same LineType as before on non-touched lines
df_ltodl = directional_lines_tab.copy().rename(columns={"NodeFrom": "FromNode", "NodeTo": "ToNode"})
df_ltodl = df_ltodl.merge(line_type_of_directional_lines_tab, on=["FromNode", "ToNode"], how="left")
df_ltodl

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Hungary,Austria,HVAC_OverheadLine
3,Italy,Austria,HVAC_OverheadLine
4,Slovenia,Austria,HVAC_OverheadLine
...,...,...,...
871,NO2,Energyhub Central,NaN
872,Great Brit.,Energyhub Central,NaN
873,Denmark,Energyhub Central,NaN
874,Netherlands,Energyhub Central,NaN


In [49]:
df_ltodl.isna().sum()

FromNode      0
ToNode        0
LineType    622
dtype: int64

In [50]:
# LineType of others can be found using df_dl but need to map two-ways

# First-way
offshr_areas_lines = df_dl.copy().drop(columns=["lineLength in km"])
mapping = offshr_areas_lines.set_index(['FromNode', 'ToNode'])['LineType']
df_ltodl['LineType'] = df_ltodl.apply(lambda row: mapping.get((row['FromNode'], row['ToNode']), row['LineType']), axis=1)

# Second-way
offshr_areas_lines = df_dl.copy().drop(columns=["lineLength in km"]).rename(columns={"FromNode": "ToNode", "ToNode": "FromNode"})
mapping = offshr_areas_lines.set_index(['FromNode', 'ToNode'])['LineType']
df_ltodl['LineType'] = df_ltodl.apply(lambda row: mapping.get((row['FromNode'], row['ToNode']), row['LineType']), axis=1)
df_ltodl

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Hungary,Austria,HVAC_OverheadLine
3,Italy,Austria,HVAC_OverheadLine
4,Slovenia,Austria,HVAC_OverheadLine
...,...,...,...
871,NO2,Energyhub Central,HVDC_OffshoreToOnshore
872,Great Brit.,Energyhub Central,HVDC_OffshoreToOnshore
873,Denmark,Energyhub Central,HVDC_OffshoreToOnshore
874,Netherlands,Energyhub Central,HVDC_OffshoreToOnshore


In [51]:
df_ltodl.isna().sum()

FromNode    0
ToNode      0
LineType    0
dtype: int64

In [52]:
line_type_of_directional_lines_tab = df_ltodl.copy()
line_type_of_directional_lines_tab

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Hungary,Austria,HVAC_OverheadLine
3,Italy,Austria,HVAC_OverheadLine
4,Slovenia,Austria,HVAC_OverheadLine
...,...,...,...
871,NO2,Energyhub Central,HVDC_OffshoreToOnshore
872,Great Brit.,Energyhub Central,HVDC_OffshoreToOnshore
873,Denmark,Energyhub Central,HVDC_OffshoreToOnshore
874,Netherlands,Energyhub Central,HVDC_OffshoreToOnshore


#### Tab: GeneratorsOfNode

In [53]:
generators_of_node_tab_public = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="GeneratorsOfNode", header=2)
generators_of_node_tab_public["unique_id"] = generators_of_node_tab_public.apply(lambda row: "".join(row["Node"] + row["Generator"]), axis=1)
generators_of_node_tab_public

,Node,Generator,unique_id
0,Bosnia H,Liginite existing,Bosnia HLiginite existing
1,Bulgaria,Liginite existing,BulgariaLiginite existing
2,Czech R,Liginite existing,Czech RLiginite existing
3,Germany,Liginite existing,GermanyLiginite existing
4,Estonia,Liginite existing,EstoniaLiginite existing
...,...,...,...
737,NO1,Hydrogen CCGT,NO1Hydrogen CCGT
738,NO2,Hydrogen CCGT,NO2Hydrogen CCGT
739,NO3,Hydrogen CCGT,NO3Hydrogen CCGT
740,NO4,Hydrogen CCGT,NO4Hydrogen CCGT


In [54]:
additional_from_open = pd.read_excel("Data_OpenEMPIRE/Sets.xlsx", sheet_name="GeneratorsOfNode", header=2)
additional_from_open["unique_id"] = additional_from_open.apply(lambda row: "".join(row["Node"] + row["Generator"]), axis=1)
additional_from_open = additional_from_open[~additional_from_open["unique_id"].isin(generators_of_node_tab_public["unique_id"].values)]
additional_from_open

,Node,Generator,unique_id
604,Croatia,Geo,CroatiaGeo
613,Bosnia H,Hydro run-of-the-river,Bosnia HHydro run-of-the-river
628,Helgolander Bucht,Wind offshore grounded,Helgolander BuchtWind offshore grounded
629,Nordsoen,Wind offshore grounded,NordsoenWind offshore grounded
631,Sorlige Nordsjo I,Wind offshore grounded,Sorlige Nordsjo IWind offshore grounded
632,Sorlige Nordsjo II,Wind offshore grounded,Sorlige Nordsjo IIWind offshore grounded
656,Helgolander Bucht,Wind offshore floating,Helgolander BuchtWind offshore floating
657,Nordsoen,Wind offshore floating,NordsoenWind offshore floating
659,Sorlige Nordsjo I,Wind offshore floating,Sorlige Nordsjo IWind offshore floating
660,Sorlige Nordsjo II,Wind offshore floating,Sorlige Nordsjo IIWind offshore floating


Remove offshore wind rows (incl. but different naming convention)

In [55]:
wo_values = ["Wind offshore grounded", "Wind offshore floating"]
additional_from_open = additional_from_open[~additional_from_open["Generator"].isin(wo_values)]
generators_of_node_tab = pd.concat([generators_of_node_tab_public, additional_from_open], ignore_index=True)
generators_of_node_tab = generators_of_node_tab.drop(columns=["unique_id"]).reset_index(drop=True)
generators_of_node_tab

,Node,Generator
0,Bosnia H,Liginite existing
1,Bulgaria,Liginite existing
2,Czech R,Liginite existing
3,Germany,Liginite existing
4,Estonia,Liginite existing
...,...,...
757,NO1,Nuclear
758,NO2,Nuclear
759,NO3,Nuclear
760,NO4,Nuclear


Remove and update offshore wind areas

In [56]:
generators_of_node_tab = generators_of_node_tab[~generators_of_node_tab["Node"].isin(NODES_TO_BE_REMOVED)]
generators_of_node_tab = generators_of_node_tab[~generators_of_node_tab["Node"].isin(offshr_areas)].reset_index(drop=True)
generators_of_node_tab

,Node,Generator
0,Bosnia H,Liginite existing
1,Bulgaria,Liginite existing
2,Czech R,Liginite existing
3,Estonia,Liginite existing
4,Greece,Liginite existing
...,...,...
538,Latvia,Nuclear
539,Macedonia,Nuclear
540,Portugal,Nuclear
541,Serbia,Nuclear


**NOTE: For Scenario generation EMPIRE needs both techs to be included for all offshore wind nodes!**

In [57]:
grounded_areas = pd.read_csv("Output/offshore_capacities_base.csv")[["Node"]].drop_duplicates(subset='Node', keep="first", ignore_index=True)
grounded_areas["Generator"] = "Wind offshore grounded"
floating_areas = grounded_areas.copy()
floating_areas["Generator"] = "Wind offshore floating"
floating_areas

,Node,Generator
0,Moray Firth,Wind offshore floating
1,Firth of Forth,Wind offshore floating
2,Dogger Bank,Wind offshore floating
3,Hornsea,Wind offshore floating
4,Outer Dowsing,Wind offshore floating
5,Norfolk,Wind offshore floating
6,East Anglia,Wind offshore floating
7,Borssele,Wind offshore floating
8,Hollandsee Kust,Wind offshore floating
9,Helgoländer Bucht,Wind offshore floating


In [58]:
generators_of_node_tab = pd.concat([generators_of_node_tab, grounded_areas, floating_areas], ignore_index=True)
generators_of_node_tab

,Node,Generator
0,Bosnia H,Liginite existing
1,Bulgaria,Liginite existing
2,Czech R,Liginite existing
3,Estonia,Liginite existing
4,Greece,Liginite existing
...,...,...
600,Nordavind C,Wind offshore floating
601,Vestavind E,Wind offshore floating
602,Sørvest E,Wind offshore floating
603,Sørvest A,Wind offshore floating


Remove hydrogen for current version to be used

In [59]:
generators_of_node_tab = generators_of_node_tab[generators_of_node_tab["Generator"] != "Hydrogen OCGT"]
generators_of_node_tab = generators_of_node_tab[generators_of_node_tab["Generator"] != "Hydrogen CCGT"].reset_index(drop=True)
generators_of_node_tab

,Node,Generator
0,Bosnia H,Liginite existing
1,Bulgaria,Liginite existing
2,Czech R,Liginite existing
3,Estonia,Liginite existing
4,Greece,Liginite existing
...,...,...
552,Nordavind C,Wind offshore floating
553,Vestavind E,Wind offshore floating
554,Sørvest E,Wind offshore floating
555,Sørvest A,Wind offshore floating


#### Tab: GeneratorsOfTechnology

Keep as is

### Write to Excel and update sheets

Sheets to be updated and corresponding dataframe

In [66]:
SHEET_TO_DF_AND_ROW = dict({
    "Nodes": [nodes_tab, 2],
    "OffshoreNodes": [offshore_nodes_tab, 2],
    "DirectionalLines": [directional_lines_tab, 4],
    "LineTypeOfDirectionalLines": [line_type_of_directional_lines_tab, 4],
    "GeneratorsOfNode": [generators_of_node_tab, 4]
})

In [67]:
# Clean and overwrite existing input_file from original
import shutil

source = "Data_EMPIRE-Public/Sets.xlsx"
destination = "EMPIRE_input/Sets.xlsx"

shutil.copy(source, destination)

'EMPIRE_input/Sets.xlsx'

In [68]:
import openpyxl as ox

def update_spreadsheet(path:str, sheet_name:str, _df : pd.DataFrame, startcol:int=1, startrow:int=4):
    wb = ox.load_workbook(path)
    try: 
        ws=wb[sheet_name]
    except KeyError:
        ws = wb.create_sheet(sheet_name)
    
    # Write headers
    idx = 0
    for col in _df.columns:
        ws.cell(row = startrow - 1, column = startcol + idx).value = col
        idx += 1

    for row in range(0, _df.shape[0]): #For each row in the dataframe
        for col in range(0, _df.shape[1]): #For each column in the dataframe
            ws.cell(row = startrow + row, column = startcol + col).value = _df.iat[row, col]
    wb.save(path)

In [69]:
#Call the defined function
for sheet, df_and_row in SHEET_TO_DF_AND_ROW.items():
    df = df_and_row[0]
    row = df_and_row[1]
    update_spreadsheet(destination, sheet, df, startrow=row)